In [1]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import simpy
import random

# Constants
AVG_TIME_BETWEEN_ARRIVALS = 5 * 60  # Average time between vessel arrivals in minutes (5 hours)
CONTAINERS_PER_VESSEL = 150
TIME_TO_MOVE_CONTAINER = 3  # Time for a crane to move one container in minutes
TIME_TO_DROP_CONTAINER = 6  # Time for a truck to drop off one container in minutes
SIMULATION_TIME = 24 * 60  # Total simulation time in minutes (24 hours)

class ContainerTerminal:
    def __init__(self, env):
        self.env = env
        self.berths = simpy.Resource(env, capacity=2)
        self.quay_cranes = simpy.Resource(env, capacity=2)
        self.trucks = simpy.Resource(env, capacity=3)
        self.vessel_count = 0

    def vessel_arrival(self):
        while True:
            yield self.env.timeout(random.expovariate(1.0 / AVG_TIME_BETWEEN_ARRIVALS))
            self.vessel_count += 1
            self.env.process(self.handle_vessel(f'Vessel {self.vessel_count}'))

    def handle_vessel(self, vessel_name):
        arrival_time = self.env.now
        print(f'{arrival_time}: {vessel_name} arrives at the terminal')

        with self.berths.request() as berth_request:
            yield berth_request
            berth_time = self.env.now
            print(f'{berth_time}: {vessel_name} is berthing')

            for i in range(CONTAINERS_PER_VESSEL):
                yield self.env.process(self.unload_container(vessel_name))

            departure_time = self.env.now
            print(f'{departure_time}: {vessel_name} leaves the terminal')

    def unload_container(self, vessel_name):
        with self.quay_cranes.request() as crane_request:
            yield crane_request
            start_unload_time = self.env.now
            print(f'{start_unload_time}: {vessel_name} starts unloading container')

            with self.trucks.request() as truck_request:
                yield truck_request
                start_transport_time = self.env.now
                print(f'{start_transport_time}: {vessel_name} loading container onto truck')

                yield self.env.timeout(TIME_TO_MOVE_CONTAINER)
                unload_time = self.env.now
                print(f'{unload_time}: {vessel_name} unloaded container onto truck')

                yield self.env.timeout(TIME_TO_DROP_CONTAINER)
                return_time = self.env.now
                print(f'{return_time}: Truck returned to terminal')


In [12]:
random.seed(42)
env = simpy.Environment()
terminal = ContainerTerminal(env)
env.process(terminal.vessel_arrival())
env.run(until=SIMULATION_TIME)

306.0180861824403: Vessel 1 arrives at the terminal
306.0180861824403: Vessel 1 is berthing
306.0180861824403: Vessel 1 starts unloading container
306.0180861824403: Vessel 1 loading container onto truck
309.0180861824403: Vessel 1 unloaded container onto truck
313.61673789526196: Vessel 2 arrives at the terminal
313.61673789526196: Vessel 2 is berthing
313.61673789526196: Vessel 2 starts unloading container
313.61673789526196: Vessel 2 loading container onto truck
315.0180861824403: Truck returned to terminal
315.0180861824403: Vessel 1 starts unloading container
315.0180861824403: Vessel 1 loading container onto truck
316.61673789526196: Vessel 2 unloaded container onto truck
318.0180861824403: Vessel 1 unloaded container onto truck
322.61673789526196: Truck returned to terminal
322.61673789526196: Vessel 2 starts unloading container
322.61673789526196: Vessel 2 loading container onto truck
324.0180861824403: Truck returned to terminal
324.0180861824403: Vessel 1 starts unloading con